## The gioi di dong



In [ ]:
import bs4
import pandas as pd
import requests

mainURL="https://www.thegioididong.com"
url="https://www.thegioididong.com/Category/FilterProductBox?c=42&o=9&pi="

names=[]
def get_link_from_json(url):
  headers={'User-Agent':'PostmanRuntime/7.29.0'}
  page = requests.get(url,headers=headers)
  listproducts=page.json()['listproducts']
  soup=bs4.BeautifulSoup(listproducts,"html.parser")
  links = soup.findAll('a', class_='main-contain')
  global names
  names=names+[link.h3.string.strip() for link in links]
  return [link.get('href') for link in links]


allLinks=[]
for i in range(5):
  allLinks+=get_link_from_json(url+str(i))


In [ ]:

def getSoup(url):
  headers={'User-Agent':'PostmanRuntime/7.29.0'}
  page = requests.get(url,headers=headers)
  return bs4.BeautifulSoup(page.text,"html.parser")


In [ ]:
tgdd=[]
for link in allLinks:
  soup=getSoup(mainURL+link)
  row=[]
  row.append(names.pop(0))
  specs=soup.find('ul', class_='parameter__list')
  lis=specs.findAll('li')
  for li in lis[:9]:
    content=""
    for span in li.span:
      content+=span.string
    row.append(content)
  row.append(soup.find('p',class_='box-price-present').string)
  tgdd.append(row)
tgdd


In [ ]:
df = pd.DataFrame(tgdd, columns =["Name","Screen","OS","Rear camera","Front camera","Chip","RAM","ROM","SIM","Batery","Price"]) 
df.to_csv("thegioididong.csv")

## FPT shop

In [ ]:
import lxml
mainURL="https://fptshop.com.vn/"
soup = bs4.BeautifulSoup(open("fptshop.html", encoding="utf8"), features="lxml")

links = soup.findAll('a',class_="cdt-product__name")
links=[link.get('href') for link in links]


In [ ]:
url="https://fptshop.com.vn/dien-thoai/tecno-spark-7t"
headers={'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36',
         'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
         }
page = requests.get(url,headers=headers)
soup= bs4.BeautifulSoup(page.text,"html.parser")

table=soup.find('table',class_="st-pd-table")
print(table)

None


## CellphoneS 

In [ ]:
mainURL="https://www.thegioididong.com"


def get_link_from_json_CPS(url):
  headers={'User-Agent':'PostmanRuntime/7.29.0'}
  page = requests.get(url,headers=headers)
  listproducts=page.json()
  links=[product['url'] for product in listproducts if float(product['final_price']) >0]
  return links

links=[]
for i in range(20):
  print(i)
  url="https://cellphones.com.vn/lapi/LoadMoreProductCate/index/?page={}&id=3&order=view_count2&dir=desc&fearture=flashsale_samsung".format(i)
  links+=get_link_from_json_CPS(url)
links


In [ ]:
cellphoneS=[]
for (i,link) in enumerate(links):
  try:
    print(i,link)
    row=[]
    soup=getSoup(link)
    row.append(soup.find('h1').string.strip())
    table=soup.find('table',id="tskt")
    trs=table.findAll('tr')
    ths = trs[0].findAll('th')
    row.append(ths[1].string)
    ths = trs[1].findAll('th')
    if ths[0].string=="Công nghệ màn hình":
      print('if')
      row.append(ths[1].string)
      for tr in trs[2:9]:
        ths = tr.findAll('th')
        if ths[1].string:
          row.append(ths[1].string)
        else:
          row.append(ths[1].contents)
    else:
      print('else')
      row.append(None)
      for tr in trs[1:8]:
        ths = tr.findAll('th')
        if ths[1].string:
          row.append(ths[1].string)
        else:
          row.append(ths[1].contents)
    row.append(soup.find('p',class_='special-price').string)
    cellphoneS.append(row)
  except Exception as e:
    print('error: ',i, e)
    continue



In [ ]:
cellphoneS

In [ ]:
df = pd.DataFrame(cellphoneS, columns =["Name","Screen size","Screen type","Rear camera","Front camera","Chip","RAM","ROM","Batery","SIM","Price"]) 
df.to_csv("cellphoneS.csv")
df

,Name,Screen size,Screen type,Rear camera,Front camera,Chip,RAM,ROM,Batery,SIM,Price
0,iPhone 11 64GB I Chính hãng VN/A,6.1 inches,IPS LCD,"[Camera kép 12MP:, [- Camera góc rộng: ƒ/1.8 a...","12 MP, ƒ/2.2 aperture",A13 Bionic,4 GB,64 GB,3110 mAh,Nano-SIM + eSIM,11.490.000 ₫
1,iPhone 13 Pro Max 128GB | Chính hãng VN/A,6.7 inches,OLED,"[Camera góc rộng: 12MP, ƒ/1.5 , [], Camera gó...","12MP, ƒ/2.2",Apple A15,6 GB,128 GB,"4,325mAh",2 SIM (nano‑SIM và eSIM),29.790.000 ₫
2,iPhone 12 Pro Max 128GB I Chính hãng VN/A,6.7 inches,OLED,"[Camera chính: 12 MP, f/1.6, 26mm, 1.4µm, dual...","[12 MP, f/2.2, 23mm (wide), 1/3.6"", [ SL 3D, (...",Apple A14 Bionic (5 nm),6 GB,128 GB,"Li-Ion, sạc nhanh 20W, sạc không dây 15W, USB ...",2 SIM (nano‑SIM và eSIM),27.000.000 ₫
3,Xiaomi Redmi Note 11 Pro Plus 5G,6.67 inches,AMOLED,"[Camera góc rộng: 108 MP, f/1.9, PDAF , [], C...",16 MP,MediaTek Dimensity 920 5G (6 nm),8 GB,256 GB,Li-Po 4500 mAh,2 SIM (Nano-SIM),9.200.000 ₫
4,Samsung Galaxy Note 20 Ultra 5G,6.9 inches,Dynamic AMOLED,"[108 MP, f/1.8, 26mm (wide), 1/1.33"", 0.8µm, P...","10 MP, f/2.2, 26mm (wide), 1/3.2"", 1.22µm, Dua...",Exynos 990 (7 nm+),12 GB,256 GB,"[Non-removable Li-Ion 4500 mAh battery, [ Fast...",2 SIM (Nano-SIM),18.990.000 ₫
...,...,...,...,...,...,...,...,...,...,...,...
221,Realme C35 4GB 128GB,6.6 inches,IPS LCD,"Chính 50 MP & Phụ 2 MP, 0.3 MP",8MP,Unisoc T616 8 nhân,4 GB,128 GB,5000mAh,2 SIM (Nano-SIM),4.590.000 ₫
222,Xiaomi Redmi Note 9 3G,6.53 inches,None,"[48 MP, f/1.8, 26mm (wide), 1/2.0"", 0.8µm, PDA...","13 MP, (wide)",MediaTek Helio G85,3 GB,64 GB,"5020 mAh battery, Fast charging 18W",2 SIM (Nano-SIM),3.400.000 ₫
223,Samsung Galaxy A20s,6.5 inches,None,"[- 13 MP (F1.8) + 8 MP (F2.2) + 5 MP (F2.2) , ...","[- 8 MP (F2.0) , [ - Tự động lấy nét, Quay vid...",Qualcomm Snapdragon 450,3 GB,32 GB,"4000 mAh, công nghệ sạc nhanh siêu tốc 15W",2 SIM (Nano-SIM),3.550.000 ₫
224,Vivo Y20 (2021),6.5 inches,IPS LCD,"[13 MP, f/1.8, (wide), PDAF, [ 2 MP, f/2.4, (m...","8 MP, (wide)",MediaTek Helio P35,4 GB,64 GB,Li-Po 5000 mAh,2 SIM (Nano-SIM),3.350.000 ₫


## hoanghamobile

In [ ]:
mainURL="https://hoanghamobile.com"
url="https://hoanghamobile.com/dien-thoai-di-dong?p=8"
soup=getSoup(url)
links=soup.findAll('div',class_="promote")
links=[mainURL+link.a.get('href') for link in links]




In [ ]:
for link in links:
  print(link)

https://hoanghamobile.com/dien-thoai-di-dong/samsung-galaxy-a73-chinh-hang
https://hoanghamobile.com/dien-thoai-di-dong/xiaomi-redmi-note-10-pro-8gb-128gb-chinh-hang
https://hoanghamobile.com/dien-thoai-di-dong/samsung-galaxy-z-flip3-5G-Flex-Your-Way-2022
https://hoanghamobile.com/dien-thoai-di-dong/samsung-galaxy-s-22-8gb-128gb-chinh-hang
https://hoanghamobile.com/dien-thoai-di-dong/samsung-galaxy-s22-8gb-256gb-chinh-hang
https://hoanghamobile.com/dien-thoai-di-dong/samsung-galaxy-s22-Plus-8gb-128gb-chinh-hang
https://hoanghamobile.com/dien-thoai-di-dong/samsung-galaxy-s22-Plus-8gb-256gb-chinh-hang
https://hoanghamobile.com/dien-thoai-di-dong/samsung-galaxy-s22-Ultra-8gb-128gb-chinh-hang
https://hoanghamobile.com/dien-thoai-di-dong/samsung-galaxy-s22-Ultra-12gb-256gb-chinh-hang
https://hoanghamobile.com/dien-thoai-di-dong/samsung-galaxy-s22-Ultra-12gb-512gb-chinh-hang
https://hoanghamobile.com/dien-thoai-di-dong/samsung-galaxy-z-flip3-5G-128GB-chinh-hang
https://hoanghamobile.com/dien

In [ ]:
certificate = []
runtime = []
genre = []
for movie in soup.findAll('p',class_='text-muted'):
  if ((not movie.findAll('span',class_='certificate')) &
  (not movie.findAll('span',class_='runtime')) &
  (not movie.findAll('span',class_='genre'))):
    continue
  certificate.append('' if not [ce.text for ce in movie.findAll('span',class_='certificate')] else
  [ce.text for ce in movie.findAll('span',class_='certificate')][0])
  runtime.append('' if not [rt.text for rt in movie.findAll('span',class_='runtime')] else
  [rt.text for rt in movie.findAll('span',class_='runtime')][0])
  genre.append('' if not [gr.text for gr in movie.findAll('span',class_="genre")] else
  [rt.text for rt in movie.findAll('span',class_='genre')][0])

In [ ]:
movies = soup.findAll('h3', class_='lister-item-header')
titles = [movie.find('a').text for movie in movies]
release = [rs.find('span',class_="lister-item-year text-muted unbold").text for rs in movies]
certificate = [ce.text for ce in soup.findAll('span',class_='certificate')]
runtime = [rt.text for rt in soup.findAll('span',class_='runtime')]
genre = [gr.text for gr in soup.findAll('span',class_="genre")]
rates = [rate['data-value'] for rate in soup.findAll('div',class_='inline-block ratings-imdb-rating')]

In [ ]:
pandas.DataFrame({'titles':titles, 
                  'release':release, 
                  'certificate':certificate,
                  'runtime':runtime,
                  'genre': genre,
                  'rates': rates})

,titles,release,certificate,runtime,genre,rates
0,Top Gun: Maverick,(2022),PG-13,131 min,"\nAction, Drama",8.7
1,Everything Everywhere All at Once,(2022),R,139 min,"\nAction, Adventure, Comedy",8.4
2,The Batman,(2022),PG-13,176 min,"\nAction, Crime, Drama",7.9
3,Goodfellas,(1990),R,145 min,"\nBiography, Crime, Drama",8.7
4,RRR,(2022),Not Rated,187 min,"\nAction, Drama",8.2
...,...,...,...,...,...,...
95,The Handmaiden,(2016),Not Rated,145 min,"\nDrama, Romance, Thriller",8.1
96,Casino,(1995),R,178 min,"\nCrime, Drama",8.2
97,The Usual Suspects,(1995),R,106 min,"\nCrime, Drama, Mystery",8.5
98,The Prestige,(2006),PG-13,130 min,"\nDrama, Mystery, Thriller",8.5


In [ ]:
url="https://www.thegioididong.com/Category/FilterProductBox?c=42&o=9&pi=1"
headers={'User-Agent':'PostmanRuntime/7.29.0'}
page = requests.get(url,headers=headers)
listproducts=page.json()['listproducts']
soup=bs4.BeautifulSoup(listproducts,"html.parser")

links = soup.findAll('a', class_='main-contain')
len(links)

20

In [ ]:
url="https://www.thegioididong.com/dtdd#c=42&o=9&pi=4"
headers={'User-Agent':'PostmanRuntime/7.29.0'}
page = requests.get(url,headers=headers)
soup=bs4.BeautifulSoup(page.text,"html.parser")

links = soup.findAll('a', class_='main-contain')

In [ ]:
links=[link.get('href') for link in links]
len(links)

20